In [ ]:
!pip install bertopic
!pip install bertopic[visualization]

In [ ]:
from bertopic import BERTopic

In [ ]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Reading Data

In [ ]:
df = pd.read_csv('/content/open_dmg_6.csv', sep = ';')   # open_dmg_subset_eventcode.csv  # open_dmg_2
print(df.shape)
df.head()

## Cleaning the Text

Removing stopwords. 

*Not need to clean the texts because the texts are clean*

In [ ]:
# from nltk.stem import SnowballStemmer
# from nltk.corpus import stopwords
# import nltk

# nltk.download('stopwords')

# stopwords = stopwords.words('swedish')

# def PreprocessText(document):
      
#     # Removing stopwords
#     tokens = document.split()
#     tokens = [word for word in tokens if word not in stopwords]

#     document = ' '.join(tokens)
    
#     # Stemming words
#     stemmer = SnowballStemmer("swedish")
#     document = stemmer.stem(document)
    
#     return document

In [ ]:
# df['Text_stopwords_removed'] = [PreprocessText(t) for t in df['Text']]
# df[['Text_stopwords_removed', 'Text']]

In [ ]:
text = df['Text'].tolist()

### Checking Max Length of the Sentences

In [ ]:
#!pip install transformers

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('KB/electra-small-swedish-cased-generator')
max_len = 0

# For every sentence...
for sent in text:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    
    # Update the minimum sentence length

print('Max sentence length: ', max_len)

# Topic Modeling

In [ ]:
#topic_model = BERTopic(embedding_model=model, calculate_probabilities=True, nr_topics="auto") 
#topic_model = BERTopic(embedding_model="KB/bert-base-swedish-cased", calculate_probabilities=True, nr_topics = 30) 
# model

# af-ai-center/bert-large-swedish-uncased
# KB/electra-small-swedish-cased-generator
bert_pretrained_model = ["KB/bert-base-swedish-cased", "af-ai-center/bert-large-swedish-uncased", "KB/electra-small-swedish-cased-generator"]
topic_model = BERTopic(embedding_model=bert_pretrained_model[0], calculate_probabilities=True, nr_topics=15) 

In [ ]:
topics, probs = topic_model.fit_transform(text)

In [ ]:
# Further reduce topics
new_topics, new_probs = topic_model.reduce_topics(text, topics, probabilities=probs, nr_topics=30)

In [ ]:
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} texts have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} texts are {topic_freq['Topic'].shape[0]-1} topics")

In [ ]:
topic_freq 

In [ ]:
df_sub_dat_eventcode = df.copy()
print(df_sub_dat_eventcode.shape)
# df_whole_dat.drop("Text", axis=1, inplace=True)
# df_whole_dat.shape

In [ ]:
#len(topics)
df_whole_dat['kb_sv_base_Topics'] = topics
#df_whole_dat.drop('Topics', axis=1, inplace=True)
df_whole_dat.head()

In [ ]:
df_sub_dat_eventcode.to_csv('./bert_topics_brakeSystem_subdat_eventcode_allmodels.csv',
                         decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

In [ ]:
df_sub_dat_eventcode['roberta_Topics'] = topics
#df_whole_dat.drop('Topics', axis=1, inplace=True)
df_sub_dat_eventcode.head()

# HDBSCAN - Cluster

### SentenceTransformers Sentence-embedding

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
#topic_model = BERTopic(embedding_model=model, calculate_probabilities=True, nr_topics="auto") 
#topic_model = BERTopic(embedding_model="KB/bert-base-swedish-cased", calculate_probabilities=True, nr_topics = 30) 
# model

# af-ai-center/bert-large-swedish-uncased
# KB/electra-small-swedish-cased-generator

bert_pretrained_model = ["KB/bert-base-swedish-cased", "af-ai-center/bert-large-swedish-uncased", "KB/electra-small-swedish-cased-generator"]
sbert_pretained_model = "Contrastive-Tension/BERT-Base-Swe-CT-STSb"
model_st = SentenceTransformer(bert_pretrained_model[0], device='cuda')  # stsb-xlm-r-multilingual

In [ ]:
# https://www.sbert.net/examples/applications/computing-embeddings/README.html
model_st.max_seq_length = 80

sentence_embeddings = model_st.encode(text, normalize_embeddings=True, show_progress_bar=True)
sentence_embeddings.shape
len(sentence_embeddings[0])

In [ ]:
type(sentence_embeddings)

In [ ]:
for sentence, embedding in zip(text[:2], sentence_embeddings[0:2]):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

### Testing Contrastive Tension Model

In [ ]:
!pip install transformers

In [ ]:
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel

sbert_pretained_model = "Contrastive-Tension/BERT-Base-Swe-CT-STSb"
model_ct = AutoModel.from_pretrained(sbert_pretained_model)
tokenizer_ct = AutoTokenizer.from_pretrained(sbert_pretained_model)
#model_ct_st = SentenceTransformer(model_ct, device='cuda')  

In [ ]:
# https://github.com/huggingface/transformers/blob/master/README.md
#inputs = tokenizer_ct("Hello world!", return_tensors="pt")

inputs = [tokenizer_ct(t, return_tensors="pt") for t in text]

#outputs = model_ct(**inputs)
outputs = [model_ct(**t) for t in inputs]

In [ ]:
len(outputs)
outputs[2][1].shape

In [ ]:
ct_embedding = []

for i in range(len(outputs)):
  ct_embedding.append(outputs[i][1])

len(ct_embedding)
ct_embedding[2].shape
print(type(ct_embedding))

In [ ]:
#In general you can concatenate a whole sequence of arrays along any axis:
import torch

ct_embedding = [arr.detach().numpy() for arr in ct_embedding]
ct_embedding = np.concatenate(ct_embedding, axis=0)
ct_embedding.shape

In [ ]:
ct_embedding

### Dimension Reduction

Why is dimensionality reduction always done before clustering? - https://stats.stackexchange.com/questions/256172/why-is-dimensionality-reduction-always-done-before-clustering

R: https://towardsdatascience.com/how-to-cluster-in-high-dimensions-4ef693bacc6: Graph-based clustering (Spectral, SNN-cliq, Seurat) is perhaps most robust for high-dimensional data as it uses the distance on a graph.

(From the same article as the above link shows)HDBSCAN, i.e. Hierarchical DBSCAN, is a powerful density-based clustering algorithm which is: 1) indifferent to the shape of clusters, 2) does not require the number of clusters to be specified, 3) robust with respect to clusters with different density. Further, HBDSCAN is very attractive because it has only one hyperparameter minPts which is the minimal number of points in a cluster. It is relatively fast for large data sets, detects outlying cells, and for each cell it reports a probability of assignment to a cluster.

One very popular method for visualizing document similarity is to use t-distributed stochastic neighbor embedding, t-SNE. - https://www.scikit-yb.org/en/latest/api/text/tsne.html

#### UMAP

Uniform Manifold Approximation and Projection for Dimension Reduction

In [ ]:
!pip install umap-learn

In [ ]:
import umap

In [ ]:
# https://umap-learn.readthedocs.io/en/latest/sparse.html A text analysis example
reducer = umap.UMAP(metric='cosine', random_state=42)  # hellinger does not support negative values
reducer.fit(ct_embedding) # sentence_embeddings

In [ ]:
umap_embedding = reducer.transform(ct_embedding) # sentence_embeddings
# Verify that the result of calling transform is
# idenitical to accessing the embedding_ attribute
assert(np.all(umap_embedding == reducer.embedding_))
umap_embedding.shape

In [ ]:
plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1],cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the texts', fontsize=24);

## HDBSCAN

https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

https://www.johnkraszewski.com/nlp
Two investigated algorithms were k-means clustering (Ball Tree) (22) and HDBSCAN (23). As shown in the following visualizations, the latter method was found to be much superior with regard to the cluster structure and consistency at the cost of higher complexity. Another benefit of the chosen algorithm was specifying outliers (cluster -1, dark grey).

In [ ]:
!pip install hdbscan

In [ ]:
import hdbscan
import sklearn.cluster as cluster
import time
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

#from sklearn.preprocessing import StandardScaler 

In [ ]:
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)

In [ ]:
plot_clusters(umap_embedding, hdbscan.HDBSCAN, (), {'min_cluster_size':50})

In [ ]:
plot_clusters(sentence_embeddings, hdbscan.HDBSCAN, (), {'min_cluster_size':50})

In [ ]:
plot_clusters(umap_embedding, cluster.SpectralClustering, (), {'n_clusters':6})

In [ ]:
plot_clusters(umap_embedding, cluster.DBSCAN, (), {'eps':5.25})

In [ ]:
# min_samples: We can recover some of our original clusters by explicitly providing min_samples at the original value of 15.
# Increasing the min_cluster_size to 30 reduces the number of clusters, merging some together.
# The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. 
# Leaf clustering: https://hdbscan.readthedocs.io/en/latest/parameter_selection.html
# A: https://hdbscan.readthedocs.io/en/latest/api.html

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, 
                            #min_samples=15, 
                            #gen_min_span_tree=True, 
                            #cluster_selection_method='leaf',
                            metric='l2', #  If metric is “precomputed”, X is assumed to be a distance matrix and must be square. -A
                            allow_single_cluster=True)

clusterer.fit(umap_embedding)

### Searching Grid for min_cluster_size

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
for size in range(10, 40, 5):
  clusterer = hdbscan.HDBSCAN(min_cluster_size=size)
  labels = clusterer.fit_predict(umap_embedding)

  #print((silhouette_score(umap_embedding, labels, metric='euclidean')))
  print("With size {} and metric euclidean. silhouette_score is {}".format(size, silhouette_score(umap_embedding, labels, metric='euclidean')))
  print("With size {} and metric cosine. silhouette_score is {}".format(size, silhouette_score(umap_embedding, labels, metric='cosine')))
  #print((silhouette_score(umap_embedding, labels, metric='cosine')))

In [ ]:
for size in range(10, 40, 5):
  clusterer = hdbscan.HDBSCAN(min_cluster_size=size)
  labels = clusterer.fit_predict(sentence_embeddings)

  print((silhouette_score(sentence_embeddings, labels, metric='euclidean')))
  print((silhouette_score(sentence_embeddings, labels, metric='cosine')))

In [ ]:
color_palette = sns.color_palette('Paired', 12)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in clusterer.labels_]
cluster_member_colors = [sns.desaturate(x, p) for x, p in
                         zip(cluster_colors, clusterer.probabilities_)]
#cluster_colors
plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=cluster_colors, **plot_kwds)

In [ ]:
print(clusterer.labels_.max())

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(clusterer.labels_)) - (1 if -1 in clusterer.labels_ else 0)
print(n_clusters_)

n_noise_ = list(clusterer.labels_).count(-1)
print(n_noise_)

In [ ]:
# building cluster hierarchy
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
clusterer.fit(sentence_embeddings)

palette = sns.color_palette()
cluster_colors = [sns.desaturate(palette[col], sat)
                  if col >= 0 else (0.5, 0.5, 0.5) for col, sat in
                  zip(clusterer.labels_, clusterer.probabilities_)]
#cluster_colors
plt.scatter(umap_embedding[:, 0], umap_embedding[:, 1], c=cluster_colors, **plot_kwds)

In [ ]:
clusterer.labels_.max()

### Silhouette Score

In [ ]:
# The silhouette_score gives the average value for all the samples.
# This gives a perspective into the density and separation of the formed clusters
print(silhouette_score(umap_embedding, clusterer.labels_, metric='cosine'))
print(silhouette_score(sentence_embeddings, clusterer.labels_, metric='cosine'))
print(silhouette_score(umap_embedding, clusterer.labels_, metric='euclidean'))
silhouette_score(sentence_embeddings, clusterer.labels_, metric='euclidean')

In [ ]:
 # Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(umap_embedding, cluster_labels)

In [ ]:
clusterer.minimum_spanning_tree_.plot(edge_cmap='viridis',
                                      edge_alpha=0.6,
                                      node_size=80,
                                      edge_linewidth=2)

In [ ]:
clusterer.labels_

## Custom Function for All Models

In [ ]:
def mycluster(text_list, bert_model, custom_mincluster_size, dimension_reduc_embedding = True, custom_max_seq_length = 35):
  
  # sentence embedding
  model_st = SentenceTransformer(bert_model, device='cuda') 
  model_st.max_seq_length = custom_max_seq_length
  sentence_embeddings = model_st.encode(text_list, normalize_embeddings=True, show_progress_bar=True)

  # dimension reduction
  reducer = umap.UMAP(random_state=42, metric='cosine')
  reducer.fit(sentence_embeddings)
  umap_embedding = reducer.transform(sentence_embeddings)

  # clustering HDBSCAN
  clusterer = hdbscan.HDBSCAN(min_cluster_size=custom_mincluster_size)
  
  if dimension_reduc_embedding == True:
    clusterer.fit(umap_embedding)
  else:
    clusterer.fit(sentence_embeddings)

  unique_value = np.unique(clusterer.labels_)

  return clusterer.labels_, unique_value


In [ ]:
def mycluster_ct(text_list, custom_mincluster_size, dimension_reduc_embedding = True):
  
  # sentence embedding
  # model_st = transformers.AutoModel.from_pretrained(bert_model)
  # model_st = SentenceTransformer(bert_model, device='cuda')  
  # model_st.max_seq_length = custom_max_seq_length
  # sentence_embeddings = model_st.encode(text_list, normalize_embeddings=True, show_progress_bar=True)

  # dimension reduction
  reducer = umap.UMAP(random_state=42, metric='cosine')
  reducer.fit(ct_embedding)
  umap_embedding = reducer.transform(ct_embedding)

  # clustering HDBSCAN
  clusterer = hdbscan.HDBSCAN(min_cluster_size=custom_mincluster_size)
  
  if dimension_reduc_embedding == True:
    clusterer.fit(umap_embedding)
  else:
    clusterer.fit(ct_embedding)

  unique_value = np.unique(clusterer.labels_)

  return clusterer.labels_, unique_value

In [ ]:
custom_min_cluster_size = 20

In [ ]:
#topic_model = BERTopic(embedding_model="KB/bert-base-swedish-cased", calculate_probabilities=True, nr_topics = 30) 
# model

# af-ai-center/bert-large-swedish-uncased
# KB/electra-small-swedish-cased-generator

bert_pretrained_model = ["KB/bert-base-swedish-cased", "af-ai-center/bert-large-swedish-uncased", "KB/electra-small-swedish-cased-generator"]
sbert_pretrained_model = "Contrastive-Tension/BERT-Base-Swe-CT-STSb"
bert_base_cluster, bert_base_unique_cluster = mycluster_sbert(text, bert_model=sbert_pretrained_model,
                                                        dimension_reduc_embedding = True,
                                                        custom_mincluster_size = custom_min_cluster_size)
#elec_cluster
bert_base_unique_cluster

In [ ]:
#topic_model = BERTopic(embedding_model="KB/bert-base-swedish-cased", calculate_probabilities=True, nr_topics = 30) 
# model

# af-ai-center/bert-large-swedish-uncased
# KB/electra-small-swedish-cased-generator

af_cluster, af_unique_cluster = mycluster(text, bert_model=bert_pretrained_model[1],
                                       custom_mincluster_size = custom_min_cluster_size)
#af_cluster
af_unique_cluster

In [ ]:
elec_cluster, elec_unique_cluster = mycluster(text, bert_model=bert_pretrained_model[2],
                                                 custom_mincluster_size = custom_min_cluster_size)

elec_unique_cluster

In [ ]:
# ct model
ct_cluster, ct_unique_cluster = mycluster_ct(text, dimension_reduc_embedding = True,
                                             custom_mincluster_size = 40)
#elec_cluster
ct_unique_cluster

In [ ]:
# df['af_cluster'] = af_cluster
# df['elec_cluster'] = elec_cluster
df['ct_cluster'] = ct_cluster
df

# KeyBERT

https://github.com/MaartenGr/KeyBERT

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
# https://github.com/MaartenGr/KeyBERT/issues/24
bert_pretrained_model = ["xlm-r-bert-base-nli-stsb-mean-tokens", # multilingual model
                         "KB/bert-base-swedish-cased", 
                         "af-ai-center/bert-large-swedish-uncased", 
                         "KB/electra-small-swedish-cased-generator"]
kw_model = KeyBERT(bert_pretrained_model[1])

In [ ]:
kw_model.extract_keywords(text, stop_words=None, keyphrase_ngram_range=(1, 7), use_maxsum=True, top_n=1)

In [ ]:
df.to_csv('./bert_topics_brakeSystem_models.csv',
          decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

# tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans
# Clustering documents with TFIDF and KMeans
# 
tfidf = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    max_features = 8000
)
tfidf.fit(text)
text_tfidf = tfidf.transform(text)

In [ ]:
from sklearn.cluster import MiniBatchKMeans

def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1, 2)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(text_tfidf, 50)

In [ ]:
clusters = MiniBatchKMeans(n_clusters=14, init_size=1024, batch_size=2048, random_state=20).fit_predict(text_tfidf)

In [ ]:
df['cluster_tfidf'] = clusters
df

In [ ]:
df.to_csv('./bert_topics_brakeSystem_model_tfidf.csv',
          decimal = ",", header = True, index = False, sep = ";", encoding = "utf-8-sig")

In [ ]:
while True:pass